<a href="https://colab.research.google.com/github/Fredf23/TatChim/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22_%22Transformer_%D1%81_%D0%BE%D0%BF%D1%82%D0%B8%D0%BC%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D0%B5%D0%B9_ipynb%22%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Обоснование изменений
1. Увеличение количества слоев и размера скрытого слоя: Увеличивает способность модели обучаться сложным зависимостям.
2. Dropout и L2-регуляризация: Снижают переобучение.
3. Использование AdamW оптимизатора: Сочетает преимущества Adam и L2-регуляризации.
4. Увеличение количества эпох: Обеспечивает более длительное обучение модели для достижения лучших результатов.

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler

# Загрузка данных из Google Drive
#from google.colab import drive
#drive.mount('/content/drive')

# Путь к файлу в Google Drive
#excel_file_path = '/content/drive/My Drive/X_bp.xls'  # Убедитесь, что путь к файлу корректен
csv_file_path = '/content/drive/MyDrive/X_bp.csv'

# Загрузка данных из Excel файла
#sheet_name = 'Sheet1'  # Замените на правильное имя листа
#data = pd.read_excel(excel_file_path, sheet_name=sheet_name)

# Сохранение данных в CSV файл
#data.to_csv(csv_file_path, index=False)
#print(f'Data has been converted from Excel to CSV and saved to {csv_file_path}')

# Загрузка данных из CSV файла
data = pd.read_csv(csv_file_path)

# Разделение данных на входные и целевые признаки
input_features = data[['Соотношение матрица-наполнитель', 'Плотность, кг/м3', 'Количество отвердителя, м.%',
                       'Содержание эпоксидных групп,%_2', 'Потребление смолы, г/м2']]
target_features = data[['модуль упругости, ГПа', 'Температура вспышки, С_2', 'Поверхностная плотность, г/м2',
                        'Модуль упругости при растяжении, ГПа', 'Прочность при растяжении, МПа']]

# Разделение на тренировочный и тестовый наборы
#X_train, X_test, y_train, y_test = train_test_split(input_features, target_features, test_size=0.2, random_state=42)

mid_index = len(data) // 2
train_indices = list(range(0, mid_index - len(data) // 5)) + list(range(mid_index + len(data) // 5, len(data)))
test_indices = list(range(mid_index - len(data) // 5, mid_index + len(data) // 5))

X_train = input_features.iloc[train_indices]
y_train = target_features.iloc[train_indices]
X_test = input_features.iloc[test_indices]
y_test = target_features.iloc[test_indices]

# Нормализация данных
scaler_X = MinMaxScaler ()
scaler_y = MinMaxScaler ()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

# Преобразование данных в тензоры
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32)

# Определение модели
class TransformerModel(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim, nheads, num_encoder_layers, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=nheads, dim_feedforward=hidden_dim, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_encoder_layers, enable_nested_tensor=False)
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.fc4 = nn.Linear(hidden_dim, output_dim)
        self.fc5 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.transformer_encoder(self.fc1(x).unsqueeze(0)).squeeze(0)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

input_dim = X_train_tensor.shape[1]
output_dim = y_train_tensor.shape[1]
hidden_dim = 192# Изменение размера скрытого слоя
nheads = 4  # Количество голов должно делить hidden_dim нацело
num_encoder_layers = 8
dropout_rate = 0.1

model = TransformerModel(input_dim, output_dim, hidden_dim, nheads, num_encoder_layers, dropout_rate)

# Определение функции потерь и оптимизатора
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.95, weight_decay=0)

# Обучение модели
num_epochs = 150

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Оценка модели

model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

# Преобразование предсказаний и реальных значений обратно из нормализованного масштаба
test_outputs_unscaled = scaler_y.inverse_transform(test_outputs.numpy())
y_test_unscaled = scaler_y.inverse_transform(y_test_tensor.numpy())

# Создание DataFrame для вывода предсказанных и реальных значений
results_df = pd.DataFrame({
    'Predicted Модуль упругости, ГПа': test_outputs_unscaled[:, 0],
    'Actual Модуль упругости, ГПа': y_test_unscaled[:, 0],
    'Predicted Температура вспышки, С_2': test_outputs_unscaled[:, 1],
    'Actual Температура вспышки, С_2': y_test_unscaled[:, 1],
    'Predicted Поверхностная плотность, г/м2': test_outputs_unscaled[:, 2],
    'Actual Поверхностная плотность, г/м2': y_test_unscaled[:, 2],
    'Predicted Модуль упругости при растяжении, ГПа': test_outputs_unscaled[:, 3],
    'Actual Модуль упругости при растяжении, ГПа': y_test_unscaled[:, 3],
    'Predicted Прочность при растяжении, МПа': test_outputs_unscaled[:, 4],
    'Actual Прочность при растяжении, МПа': y_test_unscaled[:, 4]
})

# Округление значений до десятых
results_df = results_df.round(1)

# Вывод графической таблицы
from IPython.display import display
display(results_df)

Epoch [10/150], Loss: 0.0672
Epoch [20/150], Loss: 0.0522
Epoch [30/150], Loss: 0.0414
Epoch [40/150], Loss: 0.0340
Epoch [50/150], Loss: 0.0353
Epoch [60/150], Loss: 0.0338
Epoch [70/150], Loss: 0.0323
Epoch [80/150], Loss: 0.0327
Epoch [90/150], Loss: 0.0316
Epoch [100/150], Loss: 0.0318
Epoch [110/150], Loss: 0.0314
Epoch [120/150], Loss: 0.0314
Epoch [130/150], Loss: 0.0315
Epoch [140/150], Loss: 0.0312
Epoch [150/150], Loss: 0.0311
Test Loss: 0.0308


,"Predicted Модуль упругости, ГПа","Actual Модуль упругости, ГПа","Predicted Температура вспышки, С_2","Actual Температура вспышки, С_2","Predicted Поверхностная плотность, г/м2","Actual Поверхностная плотность, г/м2","Predicted Модуль упругости при растяжении, ГПа","Actual Модуль упругости при растяжении, ГПа","Predicted Прочность при растяжении, МПа","Actual Прочность при растяжении, МПа"
0,753.500000,734.799988,286.299988,301.600006,500.600006,533.700012,73.199997,74.900002,2454.600098,2010.599976
1,754.500000,31.500000,285.799988,312.700012,500.000000,518.099976,73.099998,74.800003,2442.600098,2004.800049
2,751.400024,454.799988,288.500000,288.000000,497.000000,265.299988,73.300003,79.099998,2468.800049,2590.899902
3,747.000000,1152.900024,289.100006,257.799988,494.700012,814.700012,73.199997,78.599998,2466.100098,2748.399902
4,753.700012,540.000000,285.500000,311.799988,492.299988,761.000000,73.300003,69.400002,2432.899902,2770.600098
...,...,...,...,...,...,...,...,...,...,...
403,755.299988,536.099976,285.200012,294.899994,496.799988,569.200012,73.199997,71.699997,2436.100098,2489.100098
404,758.500000,1308.599976,287.799988,331.799988,499.000000,359.100006,73.300003,76.699997,2454.800049,1930.900024
405,759.099976,693.200012,287.700012,250.000000,499.000000,930.599976,73.300003,72.400002,2455.199951,2254.199951
406,756.500000,623.099976,286.200012,271.399994,499.399994,204.000000,73.199997,71.500000,2445.500000,2751.199951
